In [1]:
import sys
project_path = r"D:/DaiHoc/machinelearning/TLCN/DoAnTotNghiep_chat_bot/"
sys.path.append(project_path)
from source.function.utils_result import RAG
from source.search.utils_search import Qdrant_Utils
from source.rerank.utils_rerank import Rerank_Utils  
from source.model.embedding_model import Sentences_Transformer_Embedding
from source.model.extract_model import Bert_Extract
from source.model.generate_model import Gemini
from source.model.rerank_model import Cohere
from source.data.vectordb.qdrant import Qdrant_Vector
from source.core.config import Settings
from source.generate.generate import Gemini_Generate
from source.extract.utils_extract import Extract_Information
from source.schema.chatbot_querry import ChatbotQuery
setting=Settings()
gemini=Gemini(setting)
print(setting.RERANK)
cohere=Cohere(setting)
print(setting.RERANK)
bert=Bert_Extract(setting)
sentences_transformer_embedding=Sentences_Transformer_Embedding(setting)
qdrant=Qdrant_Vector(setting,sentences_transformer_embedding)
rerank_Utils=Rerank_Utils(cohere)
extract_Utils= Extract_Information(bert)
generate_Utils=Gemini_Generate(gemini,setting)
qdrant_Utils=Qdrant_Utils(qdrant, generate_Utils)
rag=RAG(generate_Utils,extract_Utils,qdrant_Utils,rerank_Utils,setting,sentences_transformer_embedding)
import pandas as pd
from tqdm import tqdm
import google.generativeai as genai 
from sklearn.metrics.pairwise import cosine_similarity
import os

hghaan/rerank_model
hghaan/rerank_model


In [2]:
print(rerank_Utils.model_rerank)

## 1.Call API System

In [3]:
import os
import pandas as pd
from tqdm import tqdm

# Đọc file CSV chứa câu hỏi và câu trả lời đã có (nếu có)
df = pd.read_csv('./data/data_processed/honnhan.csv')
df=df.head(50)
# Nếu chưa có cột 'answer_from_chatbot', thêm cột với giá trị None
if 'answer_from_chatbot' not in df.columns:
    df['answer_from_chatbot'] = None

def sinh_cau_tra_loi_batch(df_batch: pd.DataFrame) -> list[str]:
    """
    Sinh câu trả lời cho mỗi câu hỏi trong batch.
    Trả về danh sách answer tương ứng.
    """
    answers = []
    for question in tqdm(df_batch['question'], desc="Sinh trả lời"):    
        article_doc, _ = rag.get_Article_Content_Results(question)
        answers.append(article_doc)
    return answers

# Chỉ lấy các dòng chưa có câu trả lời (NaN hoặc rỗng)
df_unanswered = df[df['answer_from_chatbot'].isna() | (df['answer_from_chatbot'].str.strip() == '')]

print("==== Bắt đầu sinh trả lời cho các dòng bị thiếu ====")

# Sinh câu trả lời cho toàn bộ các dòng thiếu
df_unanswered['answer_from_chatbot'] = sinh_cau_tra_loi_batch(df_unanswered)

# Cập nhật lại vào df gốc
df.loc[df_unanswered.index, 'answer_from_chatbot'] = df_unanswered['answer_from_chatbot']

# Lưu kết quả cuối cùng
final_output_path = '../Eval_System/data/data_processed/final_response_honnhan_api.csv'
df.to_csv(final_output_path, index=False)
print(f"✅ Đã cập nhật và lưu toàn bộ vào: {final_output_path}")


==== Bắt đầu sinh trả lời cho các dòng bị thiếu ====


Sinh trả lời:   0%|          | 0/50 [00:00<?, ?it/s]

['Câu 1. Tôi có thể kết hôn với cháu ruột của thím mình không?', 'Tôi có được phép kết hôn với người có quan hệ huyết thống là cháu ruột của thím theo quy định pháp luật hôn nhân gia đình không?', 'Quy định pháp luật hiện hành về việc kết hôn giữa những người có quan hệ họ hàng, cụ thể là giữa một người và cháu ruột của thím mình?', 'Điều kiện và hạn chế pháp lý đối với việc kết hôn giữa những người có quan hệ huyết thống trong phạm vi ba đời, ví dụ như trường hợp kết hôn với cháu ruột của thím?']
None
Tìm được
Query 1 có 25 kết quả
None
Tìm được
Query 2 có 25 kết quả
None
Tìm được
Query 3 có 25 kết quả
None
Tìm được
Query 4 có 25 kết quả
Tổng số kết quả tìm được: 100
Đã thực hiện xong retrival
Số document retrival được 4
Số document khi xoá trùng 39
Đang sử dụng mô hình Cohere để xếp hạng lại tài liệu.
Số document sau khi qua rerank: 5
1 . Cô , dì , chú , cậu , bác ruột không sống chung với cháu ruột có nghĩa_vụ cấp_dưỡng cho cháu trong trường_hợp cháu chưa thành_niên hoặc cháu đã thà

Sinh trả lời:   2%|▏         | 1/50 [00:34<28:09, 34.49s/it]

['Câu 2. Anh J và chị O học tiểu học cùng nhau, anh J theo bố mẹ sang định cư tại Đan Mạch. Khi về thăm quê, anh J có gặp lại chị O, từ đó cả hai nối lại tình bạn. Sau một thời gian trao đổi, liên hệ với nhau qua điện thoại, mạng xã hội, chị O tỏ ý muốn sang định cư tại Đan Mạch và nhờ anh J giúp đỡ bằng cách đồng ý kết hôn với chị. Hai bên sẽ ly hôn sau khi chị O được nhập quốc tịch và đã sang cư trú tại Đan Mạch. Trưởng hợp này pháp luật có nghiêm cấm không và nếu J và O vẫn thực hiện thì xử lý như thế nào?', 'Hành vi kết hôn giả tạo để nhập quốc tịch Đan Mạch có bị pháp luật Việt Nam cấm không và chế tài xử lý nếu vi phạm?', 'Hôn nhân có yếu tố gian dối nhằm mục đích nhập tịch nước ngoài, cụ thể là Đan Mạch, có vi phạm quy định pháp luật Việt Nam không và hậu quả pháp lý?', 'Pháp luật Việt Nam quy định như thế nào về việc xử lý hành vi lợi dụng kết hôn để nhập quốc tịch nước ngoài (Đan Mạch) rồi ly hôn?']
None
Tìm được
Query 1 có 25 kết quả
None
Tìm được
Query 2 có 25 kết quả
None
T

...

['Câu 9. Tôi được bố mẹ để thừa kế cho 80m2 đất ở, trên đất có 01 căn nhà 05 tầng. Giấy chứng nhận quyền sử dụng đất và quyền sở hữu nhà mang tên tôi. Nay tôi định kết hôn với người vợ Philippines, sau khi kết hôn tôi sẽ sang Philippines sinh sống. Nếu trong quá trình chung sống mà nảy sinh mâu thuẫn vợ chồng mà vợ tôi đòi ly hôn và đề nghị Tòa án ở Philippines xét xử ly hôn thì tài sản của tôi có bị xác định là tài sản chung của vợ chồng không? Việc giải quyết tài sản này như thế nào?', 'Nếu tôi kết hôn với người Philippines và ly hôn ở Philippines, tài sản thừa kế riêng của tôi tại Việt Nam có bị chia không?', 'Việc phân chia tài sản là nhà đất tại Việt Nam do tôi được thừa kế, khi ly hôn với vợ người Philippines tại tòa án Philippines sẽ được giải quyết như thế nào theo pháp luật?', 'Trong trường hợp ly hôn với vợ là người Philippines tại Philippines, căn nhà 5 tầng và đất ở 80m2 tôi được thừa kế riêng ở Việt Nam có được coi là tài sản chung và bị phân chia không?']
None
Tìm được
Qu

Sinh trả lời:  98%|█████████▊| 49/50 [26:55<00:31, 31.75s/it]

['Câu 10. Cô K là công dân Việt Nam đã kết hôn với người chồng Mỹ, sau 02 năm chung sống thì hai người đã ly hôn. Việc kết hôn và ly hôn của cô K đều thực hiện tại Mỹ. Giờ cô K muốn kết hôn với người Việt Nam, do cô còn giữ quốc tịch Việt Nam, nên khi ra Ủy ban nhân dân xã để xin Giấy xác nhận tình trạng hôn nhân để cô kết hôn thì bị từ chối do Ủy ban đã cấp giấy này cho cô cách đây hơn 02 năm. Cô K hỏi, cô cần làm thủ tục gì để được kết hôn?', 'Cô K, quốc tịch Việt Nam, ly hôn ở Mỹ, muốn kết hôn tại Việt Nam cần thủ tục gì khi bị từ chối cấp Giấy xác nhận tình trạng hôn nhân?', 'Trường hợp công dân Việt Nam đã ly hôn ở nước ngoài (Mỹ), nay muốn kết hôn trong nước thì cần đáp ứng những yêu cầu pháp lý và thủ tục hành chính nào?', 'Hướng dẫn thủ tục xin cấp lại Giấy xác nhận tình trạng hôn nhân cho người Việt Nam đã ly hôn ở nước ngoài và muốn tái hôn tại Việt Nam theo quy định pháp luật hiện hành?']
None
Tìm được
Query 1 có 25 kết quả
None
Tìm được
Query 2 có 25 kết quả
None
Tìm được
Q

Sinh trả lời: 100%|██████████| 50/50 [27:27<00:00, 32.95s/it]

✅ Đã cập nhật và lưu toàn bộ vào: ../Eval_System/data/data_processed/final_response_honnhan_api.csv


### 2.Eval System

> Configure

In [4]:
setting=Settings()

In [5]:
genai.configure(api_key=gemini.key_manager.get_next_key())

> Get embedding

In [6]:
def get_gemini_embedding(text: str,
                         model_name: str = "models/text-embedding-004") -> list[float]:
    res = genai.embed_content(
        model=model_name,
        content=text
    )
    return res["embedding"]

In [ ]:
df = pd.read_csv('./data/data_processed/final_response_honnhan_api.csv')
# Lấy embedding cho Answer và answer_from_chatbot
df = df.dropna(subset=['answer', 'answer_from_chatbot'])
df['answer_embedding'] = df['answer'].apply(get_gemini_embedding)
df['answer_from_chatbot_embedding'] = df['answer_from_chatbot'].apply(get_gemini_embedding)

# Tính cosine similarity giữa Answer và answer_from_chatbot
df['cosine_similarity'] = df.apply(
    lambda row: cosine_similarity(
        [row['answer_embedding']],
        [row['answer_from_chatbot_embedding']]
    )[0][0],
    axis=1
)
# Tính trung bình của cosine similarity
average_cosine_similarity = df['cosine_similarity'].mean()

print(f"Trung bình cosine similarity giữa Answer và answer_from_chatbot: {average_cosine_similarity:.4f}")

Trung bình cosine similarity giữa Answer và answer_from_chatbot: 0.8886


: 